In [1]:
import os

os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk1.8.0_291"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [2]:
rdd = sc.textFile("data/soc-LiveJournal1Adj.txt")

In [3]:
import re
def mf(line):
    line = line.split('\t')
    if(not line[1]):
        return (int(line[0]),[])
    vs = sorted([int(v) for v in line[1].split(',')])
    return (int(line[0]),vs)


def line_to_pairs (uvs):
    u,vs = uvs
    pairs = []
    n = len(vs)
    for i in range(n):
        for j in range(i):
            pairs.append(((vs[j],vs[i]),1))
    minf = -float('inf')
    for v in vs:
        if u < v:
            pairs.append(((u,v),minf))
        else:
            pairs.append(((v,u),minf))
    return pairs
            
pairs = rdd.map(mf).flatMap(line_to_pairs)

In [4]:
pair_count = pairs.reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>0)

In [5]:
def count_to_suggest(uv_num):
    uv,num = uv_num
    u,v = uv
    return (u,(num,v)),(v,(num,u))
suggest = pair_count.flatMap(count_to_suggest).groupByKey()

In [6]:
def pick_suggest(suggest):
    u,vs = suggest
    vs = sorted(vs,key=lambda v:(-v[0],v[1]))
    vs = [v[1] for v in vs[:10]]
    return u,vs
suggest = suggest.map(pick_suggest)

In [7]:
suggest.filter(lambda y:y[0]==11).collect()[0][1]

[27552, 7785, 27573, 27574, 27589, 27590, 27600, 27617, 27620, 27667]

In [8]:
for x in [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]:
    print("%s:%s" % (x,",".join(str(x) for x in suggest.filter(lambda y:y[0]==x).collect()[0][1])))

924:439,2409,6995,11860,15416,43748,45881
8941:8943,8944,8940
8942:8939,8940,8943,8944
9019:9022,317,9023
9020:9021,9016,9017,9022,317,9023
9021:9020,9016,9017,9022,317,9023
9022:9019,9020,9021,317,9016,9017,9023
9990:13134,13478,13877,34299,34485,34642,37941
9992:9987,9989,35667,9991
9993:9991,13134,13478,13877,34299,34485,34642,37941
